In [1]:
import requests
import pandas as pd

req = requests.get("https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-donnees-statiques/records?limit=20")
wb = req.json()


In [2]:
print(req.content[:1000])

b'{"total_count": 1947, "results": [{"id_pdc_local": "FR*V75*E9015*06*2", "statut_pdc": "En service", "arrondissement": "15e Arrondissement", "id_station_local": "FR*V75*P9015*06", "id_station_itinerance": "FRV75P901506", "nom_station": "Paris | Rue Desnouettes 55", "code_insee_commune": "75115", "implantation_station": "Voirie", "nbre_pdc": "3", "date_maj": "2024-07-01", "condition_acces": "Acc\\u00e8s libre", "adresse_station": "55 Rue Desnouettes 75015 Paris", "coordonneesxy": {"lon": 2.286912, "lat": 48.83571}, "gratuit": "false", "paiement_acte": "true", "paiement_cb": "false", "paiement_autre": "true", "reservation": "false", "observations": ["Paiement autre - badge RFID Belib ou autre, application Belib, QRCode sur chaque PDC, site internet Belib", "Plus d\'information - https://belib.paris.fr"], "num_pdl": "07436179348549", "siren_amenageur": "531680445", "contact_amenageur": "assistance.technique@tevgo.fr", "nom_amenageur": "TOTALENERGIES", "nom_operateur": "TOTALENERGIES", "c

In [ ]:
import pandas as pd
import geopandas as gpd


def get_dpe_from_url(url):

    req = requests.get(url)
    wb = req.json()
    df = pd.json_normalize(wb["results"])

    dpe = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326
    )
    dpe = dpe.dropna(subset=["longitude", "latitude"])

    return dpe


dpe = get_dpe_from_url(url_api)
dpe.head(2)